## Código final consolidado: LSTM com HPO + linearidade + robustez

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
import joblib
import os

# === Função para criar dataset multi-step ===
def criar_dataset_multi_step(series, look_back=10, passo=1):
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        X.append(series[i:i+look_back])
        y.append(series[i+look_back+passo-1])
    return np.array(X), np.array(y)

# === Função geradora de modelo com look_back fixado ===
def get_build_model(look_back):
    def build_model(hp):
        inputs = Input(shape=(look_back, 1))
        lstm_out = LSTM(
            units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
            return_sequences=False
        )(inputs)

        flattened = Flatten()(inputs)
        normalized = BatchNormalization()(flattened)
        linear_weights = Dense(
            1, use_bias=False,
            kernel_regularizer=l1_l2(l1=0.01, l2=0.01),
            name="linear_combination"
        )(normalized)

        combined = Concatenate()([lstm_out, linear_weights])
        output = Dense(1)(combined)

        model = Model(inputs, output)
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.Huber(delta=1.0),
            metrics=['mae']
        )
        return model
    return build_model

# === Loop para cada horizonte de previsão ===
a3_df = pd.read_csv("A3_component.csv")
a3 = a3_df["A3"].values.reshape(-1, 1)

for passo in [1, 5, 7, 30]:
    look_back = 5 if passo <= 5 else 10
    print(f"\n🚀 Treinando LSTM para t+{passo} com look_back={look_back}")

    # Normalização
    scaler = MinMaxScaler()
    a3_scaled = scaler.fit_transform(a3)
    joblib.dump(scaler, f"scaler_A3_t{passo}.joblib")

    # Criar dataset
    X, y = criar_dataset_multi_step(a3_scaled, look_back=look_back, passo=passo)

    # Tuner
    tuner = RandomSearch(
        get_build_model(look_back),
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='tuner_results',
        project_name=f'lstm_t{passo}'
    )

    # Validação cruzada temporal
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        tuner.search(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=50,
            batch_size=32,
            callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
            verbose=0
        )
        break  # usar apenas a 1ª divisão do TimeSeriesSplit

    # Treinar melhor modelo
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # Salvar modelo
    best_model.save(f"lstm_a3_t{passo}.keras")
    print(f"✅ Modelo salvo: lstm_a3_t{passo}.keras")


import joblib
joblib.dump(scaler_a3, 'scaler_a3.joblib')